In [1]:
import pandas as pd # allows us to work with tabular data
import requests # to send the API requests to Elsevier
import json # to read the JSON data that is returned by the APIs
import pickle # pickle files are a good way to save data for reuse in Python
from datetime import datetime

In [2]:
api_key = 'your api key'

In [3]:
'''search parameters: add values to the dictionary key:value pairs below
leave any fields that you don't want to use as either 0 or '''

search_string_parameters = {
    'ISSN' : '', # Limit results to results from a journal ISSN (add as a string). Leave = '' if no value.
    'AUTHOR-NAME' : '', # Search the author name field. lastname, firstname. e.g., Noble, Safiya
    'KEY' : '', # Add a keyword to search in the text of the article
    'PUBLISHER' : '', # Add a publisher name. e.g., Springer
    'EXACTSRCTITLE' : 'journal', # Add keywords that appear in the journal, book, or conference title. e.g., Informatics
    'AFFIL' : '', # add keywords that appear in the author's institutional affiliation. e.g., to find the University of Minnesota, search for University and Minnesota
    'TITLE' : '5G', # Add keywords that appear in the article or chapter title. Can use AND, OR, and AND NOT. e.g. cat AND dog 
    'OPENACCESS' : '1', # Add 1 to limit your results to open access articles or 0 for articles that are not open access
    'start_year' : 2013, # Limit your search to items published after this year. Leave = 0 if no value. YYYY , e.g., 1995
    'end_year' : 2024 # Limit your search to items published before this year. Leave = 0 if no value. YYYY, e.g., 2020
}
def search_builder(search_string_parameters):
    ''' Concatenates a search string query formatted for the Scopus Search API.
    - search_string_parameters expects a python dictionary with keys aligned to API search fields.
    '''
    search_string = ''

    for k,v in search_string_parameters.items():
        if k == 'start_year' and v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'PUBYEAR > {v}'
        elif k == 'end_year' and v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'PUBYEAR < {v}'
        elif v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'{k}({v})'

    return search_string

def create_url(search_string):
    '''Accepts a formatted search string that will be added to the Scopus Search API URL. 
    Requires a global api_key variable.
    Formats and returns a URL to send to the Scopus Search API.'''
    
    query = f'{search_string}'
    url_template = 'https://api.elsevier.com/content/search/scopus?query={query}&apiKey={api_key}'
    full_url = url_template.format(query=query, api_key=api_key)
    return full_url

def connect_to_endpoint(full_url, params={'cursor': '*'}, next_ = '*'):
    '''Accepts API URL with ISSN, default parameters, and next page cursor;
    Sends request to Scopus API and collects JSON results for each call;
    Returns r.json() for the ['search-results'] key.'''
    
    params['cursor'] = next_
    r = requests.get(full_url, params=params)
    r.raise_for_status()
    return r.json()['search-results'], r.headers

In [4]:
search_string = search_builder(search_string_parameters)
print(search_string)
full_url = create_url(search_string)
r_json, r_headers = connect_to_endpoint(full_url)

EXACTSRCTITLE(journal) AND TITLE(5G) AND OPENACCESS(1) AND PUBYEAR > 2013 AND PUBYEAR < 2024


### 검색 결과 중 첫번째 논문만 가져오자

In [6]:
print('Title:', r_json['entry'][0]['dc:title'], 
      '\nCreator:', r_json['entry'][0]['dc:creator'], 
      '\nPublication:', r_json['entry'][0]['prism:publicationName'],
      '\nDate:', r_json['entry'][0]['prism:coverDate'],
      '\nISSN:', r_json['entry'][0]['prism:eIssn'], 
      '\nDOI:', r_json['entry'][0]['prism:doi'])

Title: Area Efficient and Linear MMIC Based Ka Band Power Amplifier for 5G Communication Systems 
Creator: Agarwal D. 
Publication: Journal of Integrated Circuits and Systems 
Date: 2023-12-28 
ISSN: 18720234 
DOI: 10.29292/jics.v18i3.727


In [7]:
dois = [entry.get('prism:doi') for entry in r_json.get('entry', [])]
print(dois[0])

10.29292/jics.v18i3.727


### 추출된 doi를 이용해 abstract을 가져와보자

In [8]:
def create_url(doi):
    """Accepts a DOI as a string that will be added to the Abstract Retrieval API URL. 
    Requires a global api_key variable.
    Formats and returns a URL to send to the Abstract Retrieval API.
    """
    doi = f'{doi}'
    url_template = 'https://api.elsevier.com/content/abstract/doi/{doi}?&apiKey={api_key}'
    full_url = url_template.format(doi=doi, api_key=api_key)
    return full_url

def connect_to_endpoint(full_url):
    '''Input full_url from create_url function;
    Send request to Scopus Abstract Retrieval API
    Returns r.json response;'''
    
    r = requests.get(full_url, headers =  {'Accept': 'application/json'})
    r.raise_for_status()
    return r.json(), r.headers

In [9]:
full_url = create_url(dois[0])

In [10]:
r_json, r_headers = connect_to_endpoint(full_url)
r_json = r_json['abstracts-retrieval-response']
print(r_json['coredata']['dc:description'])

This work presents a monolithic microwave integrated circuit (MMIC) based power amplifier (PA) with single 150 nm GaN high electron mobility transistor (HEMT) using broadband matching. For the proposed amplifier, two stage L-type biasing network is designed. In order to provide good linearity, low Q, broader bandwidth and optimized chip area, band pass filter (BPF) type matching network is proposed. The topology and parameter selection is done by performing proper load pull analysis. This helps in achieving wide band matching to maintain low Q factor. Thus the proposed amplifier is evaluated with post layout EM simulation data. For the 26 GHz band of 5G communication, proposed amplifier has a small signal gain of 11 dB for 21-27.4 GHz. The amplifier offers 33.42 dBm saturated output power and good linearity. The designed PA provides 31.35 dBm output power, 26% power added efficiency (PAE) and 29% drain efficiency (DE) at 1 dB com-pression with IP3 of 39.67 dBm for 26 GHz band. The desi